<a href="https://colab.research.google.com/github/FrancoPalavicinoG/cellia/blob/main/notebooks/12_baseline_training_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Train, Val & Test Datatsets from Google Drive

Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [2]:
# Core
import pandas as pd
import numpy as np

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, classification_report

# Explainability
import shap
import matplotlib.pyplot as plt
import seaborn as sns

Load Datasets

In [3]:
input_path = "/content/drive/MyDrive/cellia_drive/Datasets/"

Load features

In [4]:
X_train = pd.read_csv(input_path + "X_train_v2.csv")
X_val   = pd.read_csv(input_path + "X_val_v2.csv")
X_test  = pd.read_csv(input_path + "X_test_v2.csv")

Load labels

In [5]:
y_train = pd.read_csv(input_path + "y_train_v2.csv").squeeze()
y_val   = pd.read_csv(input_path + "y_val_v2.csv").squeeze()
y_test  = pd.read_csv(input_path + "y_test_v2.csv").squeeze()

In [6]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(49000, 36) (49000,)
(10500, 36) (10500,)
(10500, 36) (10500,)


### Evaluation function

In [7]:
def evaluate_model(model, X, y_true, dataset_name="Validation"):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print(f"{dataset_name} Results:")
    print(f"ROC-AUC: {roc_auc:.3f} | F1: {f1:.3f} | Precision: {precision:.3f} | Recall: {recall:.3f}")
    print("-" * 60)

    return {"roc_auc": roc_auc, "f1": f1, "precision": precision, "recall": recall}

## Model Training

### Logistic Regression (Baseline)

In [8]:
log_reg = LogisticRegression(
    max_iter=500,          # iteration numbers
    random_state=42,
    class_weight='balanced',  # class balancing
    C=1.0,                 # regularization
    penalty='l2',          # penalty type: 'l1', 'l2', 'elasticnet'
    solver='liblinear'     # solver compatible with the penalty
)


### Random Forest

In [9]:
rf = RandomForestClassifier(
    n_estimators=300,      # number of trees
    max_depth=10,          # maximum depth of each tree
    min_samples_split=2,   # minimum samples to split a node
    min_samples_leaf=1,    # minimum samples per leaf
    max_features='sqrt',   # number of features to consider per split
    random_state=42,
    class_weight='balanced'
)


### XGBoost

In [10]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

### LightGBM

In [11]:
lgbm = LGBMClassifier(
    n_estimators=300,      # number of iterations
    max_depth=6,           # maximum tree depth
    num_leaves=31,         # leaves per tree
    learning_rate=0.05,    # learning rate
    min_child_samples=20,  # minimum samples per leaf node
    subsample=0.8,         # fraction of data per tree
    colsample_bytree=0.8,  # fraction of features per tree
    random_state=42,
    scale_pos_weight=1     # class balancing if not using SMOTE
)

### MLP Tabular

In [12]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

### Train models

In [13]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm,
          "MLP": mlp,
          "XGBoost": xgb}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

Training Logistic Regression...
Validation (Logistic Regression) Results:
ROC-AUC: 0.781 | F1: 0.699 | Precision: 0.742 | Recall: 0.660
------------------------------------------------------------
Training Random Forest...
Validation (Random Forest) Results:
ROC-AUC: 0.799 | F1: 0.717 | Precision: 0.754 | Recall: 0.684
------------------------------------------------------------
Training LightGBM...
[LightGBM] [Info] Number of positive: 24485, number of negative: 24515
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025186 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5108
[LightGBM] [Info] Number of data points in the train set: 49000, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499694 -> initscore=-0.001224
[LightGBM] [Info] Start training from score -0.001224
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

### Compare models

In [14]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm,
          "MLP": mlp,
          "XGBoost": xgb}

results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)

Validation (Logistic Regression) Results:
ROC-AUC: 0.781 | F1: 0.699 | Precision: 0.742 | Recall: 0.660
------------------------------------------------------------
Validation (Random Forest) Results:
ROC-AUC: 0.799 | F1: 0.717 | Precision: 0.754 | Recall: 0.684
------------------------------------------------------------
Validation (LightGBM) Results:
ROC-AUC: 0.801 | F1: 0.722 | Precision: 0.751 | Recall: 0.694
------------------------------------------------------------
Validation (MLP) Results:
ROC-AUC: 0.729 | F1: 0.695 | Precision: 0.602 | Recall: 0.824
------------------------------------------------------------
Validation (XGBoost) Results:
ROC-AUC: 0.801 | F1: 0.721 | Precision: 0.751 | Recall: 0.693
------------------------------------------------------------


,roc_auc,f1,precision,recall
LightGBM,0.800858,0.721743,0.751444,0.694302
XGBoost,0.800846,0.721110,0.751187,0.693349
Random Forest,0.798789,0.717170,0.753940,0.683819
Logistic Regression,0.781375,0.698941,0.742288,0.660377
MLP,0.729167,0.695463,0.601670,0.823899


## Hyperparameter Tuning using RandomizedSearchCV

In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
import numpy as np

# ---------------------------
# Logistic Regression
# ---------------------------
logreg = LogisticRegression(max_iter=1000, random_state=42)

logreg_param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

logreg_search = RandomizedSearchCV(
    logreg,
    param_distributions=logreg_param_dist,
    n_iter=5,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

logreg_search.fit(X_train, y_train)
print("Best Logistic Regression params:", logreg_search.best_params_)
print("Best F1:", logreg_search.best_score_)

# ---------------------------
# Random Forest
# ---------------------------
rf = RandomForestClassifier(random_state=42)

rf_param_dist = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(5, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

rf_search = RandomizedSearchCV(
    rf,
    param_distributions=rf_param_dist,
    n_iter=5,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
print("Best Random Forest params:", rf_search.best_params_)
print("Best F1:", rf_search.best_score_)

# ---------------------------
# LightGBM
# ---------------------------
lgbm = LGBMClassifier(random_state=42)

lgbm_param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.01, 0.1),
    'n_estimators': randint(200, 1000),
    'min_child_samples': randint(10, 50),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'class_weight': [None, 'balanced']
}

lgbm_search = RandomizedSearchCV(
    lgbm,
    param_distributions=lgbm_param_dist,
    n_iter=5,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

lgbm_search.fit(X_train, y_train)
print("Best LightGBM params:", lgbm_search.best_params_)
print("Best F1:", lgbm_search.best_score_)


# ---------------------------
# XGBoost
# ---------------------------
xgb = XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_param_dist = {
    'n_estimators': randint(200, 800),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 2),
    'scale_pos_weight': [1, (y_train.value_counts()[0] / y_train.value_counts()[1])]
}

xgb_search = RandomizedSearchCV(
    xgb,
    param_distributions=xgb_param_dist,
    n_iter=5,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_search.fit(X_train, y_train)
print("Best XGBoost params:", xgb_search.best_params_)
print("Best F1:", xgb_search.best_score_)

NameError: name 'X_train' is not defined

## Optimize Hyperparameters Models

### Get the best pre-trained models directly from RandomizedSearchCV.

In [ ]:
best_logreg = logreg_search.best_estimator_
best_rf = rf_search.best_estimator_
best_lgbm = lgbm_search.best_estimator_
best_xgb = xgb_search.best_estimator_

from sklearn.metrics import classification_report, f1_score

models = {
    "Logistic Regression": best_logreg,
    "Random Forest": best_rf,
    "LightGBM": best_lgbm,
    "XGBoost": best_xgb
}

results = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("F1 Score:", f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)


=== Logistic Regression ===
F1 Score: 0.39215686274509803
              precision    recall  f1-score   support

           0       0.97      0.76      0.85       706
           1       0.26      0.78      0.39        77

    accuracy                           0.76       783
   macro avg       0.62      0.77      0.62       783
weighted avg       0.90      0.76      0.81       783

Validation (Logistic Regression) Results:
ROC-AUC: 0.859 | F1: 0.400 | Precision: 0.268 | Recall: 0.792
------------------------------------------------------------

=== Random Forest ===
F1 Score: 0.3957597173144876
              precision    recall  f1-score   support

           0       0.96      0.79      0.87       706
           1       0.27      0.73      0.40        77

    accuracy                           0.78       783
   macro avg       0.62      0.76      0.63       783
weighted avg       0.90      0.78      0.82       783

Validation (Random Forest) Results:
ROC-AUC: 0.840 | F1: 0.388 | Preci

,roc_auc,f1,precision,recall
Logistic Regression,0.859280,0.400000,0.267544,0.792208
XGBoost,0.854969,0.400000,0.270642,0.766234
Random Forest,0.840011,0.388489,0.268657,0.701299
LightGBM,0.833582,0.377622,0.258373,0.701299


### Export trained models

In [ ]:
import joblib

output_path = "/content/drive/MyDrive/cellia_drive/Models/"

import os
os.makedirs(output_path, exist_ok=True)

# Guardar cada modelo
for name, model in models.items():
    file_path = os.path.join(output_path, f"{name}.joblib")
    joblib.dump(model, file_path)
    print(f"✅ {name} saved to {file_path}")

✅ Logistic Regression saved to /content/drive/MyDrive/cellia_drive/Models/Logistic Regression.joblib
✅ Random Forest saved to /content/drive/MyDrive/cellia_drive/Models/Random Forest.joblib
✅ LightGBM saved to /content/drive/MyDrive/cellia_drive/Models/LightGBM.joblib
✅ XGBoost saved to /content/drive/MyDrive/cellia_drive/Models/XGBoost.joblib
